In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import h5py

In [49]:
demographic_keys = ['age']

EHR_keys_numerical = ['AST', 'Alb', 'BNP', 'BUN', 'Cr', 'HCO2', 'Hgb' , 'INR', 'K', 'Na', 'PEEP', 'PTT', 'Plat', 'Plt', 'RRobs', 'RRset', 'TP', 'Tbili', 'Vtobs', 'Vtset', 'WBC', 'dbp', 'fi02', 'hr', 'lactate', 'lipase', 'mAirP', 'pf', 'pf_calc', 'pCO2', 'pH', 'pO2', 'phenyl', 'rr', 'sbp', 'sp02', 'temp', 'trop', 'urine_out', 'dobu', 'vasso', 'dopa', 'epi', 'ffp_transf','iv_in','norepi', 'plt_transf', 'rass', 'rbc_transf']

EHR_keys_categorical = ['alert', 'dialysis',   'hfnc', 'invasive',  'noninvasive', 'oriented', 'sedated', 'supl', 'unresponsive','gcs_eye', 'gcs_motor', 'gcs_total', 'gcs_verbal', 'mil']

EHR_label = ['ards']
EHR_identifiers = ['patient_id', 'time', 'encounter_id']

EHR_keys = EHR_keys_numerical + EHR_keys_categorical

In [50]:
# Load the CSV file
df = pd.read_csv(f'data/processed_ARDS_data.csv')

# Sort by patient_id and time
df = df.sort_values(['patient_id', 'time'])
df.head()

,patient_id,encounter_id,time,AST,Alb,BNP,BUN,Cr,HCO2,Hgb,...,sedated,sp02,supl,temp,total_in,total_out,trop,unresponsive,urine_out,vasso
0,100001001,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.0,1.0,98.1,NaN,NaN,NaN,NaN,NaN,NaN
1,100001001,1,0.800000,23.0,3.8,NaN,17.0,0.75,21.0,11.6,...,NaN,97.0,1.0,98.1,NaN,NaN,NaN,NaN,NaN,NaN
2,100001001,1,0.983333,23.0,3.8,NaN,17.0,0.75,21.0,11.6,...,NaN,98.0,1.0,98.1,NaN,NaN,NaN,NaN,NaN,NaN
3,100001001,1,1.166667,23.0,3.8,NaN,17.0,0.75,21.0,11.6,...,0.0,98.0,1.0,98.1,NaN,NaN,NaN,0.0,NaN,NaN
4,100001001,1,3.350000,23.0,3.8,NaN,17.0,0.75,21.0,11.6,...,0.0,98.0,1.0,98.1,NaN,NaN,NaN,0.0,NaN,NaN


In [51]:
df[EHR_keys] = df.groupby('patient_id')[EHR_keys].ffill()
df[EHR_keys] = df.groupby('patient_id')[EHR_keys].bfill()

In [53]:
scaler = StandardScaler()
df[EHR_keys_numerical] = scaler.fit_transform(df[EHR_keys_numerical])

In [54]:
# Create an HDF5 file
with h5py.File(r'data/ARDS_processed.h5', 'w') as hf:
    for patient_id, group in df.groupby('patient_id'):
        patient_group = hf.create_group(str(patient_id))
        for column in group.columns:
            patient_group.create_dataset(column, data=group[column])